这是hot100的第12题：最小覆盖子串 给你一个字符串 s 、一个字符串 t 。返回 s 中涵盖 t 所有字符的最小子串。如果 s 中不存在涵盖 t 所有字符的子串，则返回空字符串 "" 。

In [ ]:
class Solution:
    def minWindow(self, s: str, t: str) -> str:
        if len(s) < len(t):
            return ""
        ans = ""
        d = Counter(t)
        dd = Counter()
        j = 0
        for i in range(len(s)):
            if s[i] in d:
                dd[s[i]] += 1
            while dd >= d:
                if not ans or len(ans) > i - j + 1:
                    ans = s[j:i+1]
                if s[j] in d:
                    dd[s[j]] -= 1
                j += 1  
        return ans

失败了很多次啊，看了提示才知道，一是错在使用了字典的方式进行比较包含关系，但是判断条件太复杂了,包含关系没弄懂，思路是对的，就是使用滑动窗口进行移动求解，不知道有个Counter()可以求两串字符串的包含关系，二是知道有counter可以比较的时候，才想着滑动窗口这怎么使用，不用急着把j往后移动，先到ij之间包含的时候，再移动j，每移动一次，就判断包含关系，然后求最小的那个保存起来，当不满足包含了，就把把后指针移动，直到移动到最后一个，结束
学到了：

1.counter()可以进行包含关系的计算，counter(str/list)都可以，生成的是一个字典，c1>=c2意味着c1包含c2

2.这个的时间复杂度在O(∣Σ∣m+n)，其中∣Σ∣的意思是在匹配的过程中，counter会对字符串进行比较，而字符串集合的键有数量限制，为∣Σ∣，这里是英文字母52个，而每次移动窗口都要比较一次，所以是∣Σ∣*m次，m为总字符串的长度

In [ ]:
class Solution:
    def minWindow(self, s: str, t: str) -> str:
        ans_left, ans_right = -1, len(s)
        cnt = defaultdict(int)  # 比 Counter 更快
        for c in t:
            cnt[c] += 1
        less = len(cnt)  # 有 less 种字母的出现次数 < t 中的字母出现次数

        left = 0
        for right, c in enumerate(s):  # 移动子串右端点
            cnt[c] -= 1  # 右端点字母移入子串
            if cnt[c] == 0:
                # 原来窗口内 c 的出现次数比 t 的少，现在一样多
                less -= 1
            while less == 0:  # 涵盖：所有字母的出现次数都是 >=
                if right - left < ans_right - ans_left:  # 找到更短的子串
                    ans_left, ans_right = left, right  # 记录此时的左右端点
                x = s[left]  # 左端点字母
                if cnt[x] == 0:
                    # x 移出窗口之前，检查出现次数，
                    # 如果窗口内 x 的出现次数和 t 一样，
                    # 那么 x 移出窗口后，窗口内 x 的出现次数比 t 的少
                    less += 1
                cnt[x] += 1  # 左端点字母移出子串
                left += 1
        return "" if ans_left < 0 else s[ans_left: ans_right + 1]

这部分的代码优化在于不比较counter了，而是使用了新的数据结构defaultdict，这个字典有默认值，可以得到字典的长度，我们将t字符串变成defaultdict，然后将初始答案的左右端点放在-1和len(s)，然后用双指针对s进行遍历，减去dict中的值，当dict的长度为0时，说明内部已经包含了我们想要的字符串，然后大小比较；比较完成之后，移动左指针，当less为0时，就不进行比较